In [11]:
import csv
import pandas as pd

In [17]:
#Loads all datafiles into dataframes of the same name.  Not sure the with open statements are necessary.
filenames = ['colors.csv', 'inventories.csv', 'inventory_parts.csv', 'inventory_sets.csv', 'part_categories.csv', 'parts.csv', 'sets.csv', 'themes.csv']

with open(filenames[0]) as datafile:
    colors = pd.read_csv(datafile)
with open(filenames[1]) as datafile:
    inventories = pd.read_csv(datafile)
with open(filenames[2]) as datafile:
    inventory_parts = pd.read_csv(datafile)
with open(filenames[3]) as datafile:
    inventory_sets = pd.read_csv(datafile)
with open(filenames[4]) as datafile:
    part_categories = pd.read_csv(datafile)
with open(filenames[5]) as datafile:
    parts = pd.read_csv(datafile)
with open(filenames[6]) as datafile:
    sets = pd.read_csv(datafile)
with open(filenames[7]) as datafile:
    themes = pd.read_csv(datafile)

In [18]:
#Add theme info to rows in sets dataframe
sets = pd.merge(sets, themes, left_on = 'theme_id', right_on = 'id')
sets = sets.drop('id', axis = 1) #extra column from merge
#sets = sets.set_index(['set_num']) #rename the index to the set id, maybe not useful
sets.columns = ['set_num', 'setname', 'year', 'theme_id', 'num_parts', 'theme_name', 'parent_id'] #slightly inefficient way to rename the name column

In [19]:
inventory_parts = pd.merge(inventory_parts, colors, left_on = 'color_id', right_on = 'id')

In [20]:
inventory_parts = inventory_parts.drop('id', axis = 1)
inventory_parts.columns=['inventory_id', 'part_num', 'color_id', 'quantity', 'is_spare', 'color_name', 'rgb', 'is_trans']

In [21]:
inventories = inventories.set_index('id')

In [22]:
#Attempt to add the number of parts of each color to the inventories dataframe.
#This takes a very long time.
inventoryids = inventory_parts['inventory_id'].unique() #Gets the unique list of inventory_parts inventory ids
colornames = colors['name'].unique() #same for color ids
#initialize color column to 0
for name in colornames:
    inventories[name]=0
groupedinventory = inventory_parts.groupby('inventory_id') #Generates a collection of little dfs, one for each id
for k in inventoryids:
    dummy = groupedinventory.get_group(k)
    colornames = dummy['color_name'].unique() #same for color ids
    colorgroups = dummy.groupby('color_name') #groups each little df by color_name
    #counts the number of parts of each color, and inserts it to the appropriate entry in inventory sets
    for name in colornames:
        try: 
            inventories.loc[k,name] = int(colorgroups.get_group(name)['quantity'].sum())
        except KeyError:
            pass #have the code ignore all KeyErrors and continue

In [24]:
forprediction = pd.merge(inventories, sets, left_on = 'set_num', right_on ='set_num')

In [25]:
forprediction

,version,set_num,Unknown,Black,Blue,Green,Dark Turquoise,Red,Dark Pink,Brown,...,Trans Fire Yellow,Trans Light Royal Blue,Reddish Lilac,[No Color],setname,year,theme_id,num_parts,theme_name,parent_id
0,1,7922-1,0,1,0,0,0,0,0,0,...,0,0,0,0,McDonald's Sports Set Number 6 - Orange Vest S...,2004,460,4,Gravity Games,458.0
1,1,3931-1,0,1,0,1,0,0,0,0,...,0,0,0,0,Emma's Splash Pool,2012,494,43,Friends,NaN
2,1,6942-1,0,4,5,0,0,0,0,0,...,0,0,0,0,Zo Weevil,1999,134,20,Insectoids,126.0
3,1,5158-1,0,0,0,2,0,0,0,0,...,0,0,0,0,"T-Junction, Circle Plates",1990,454,2,Town,443.0
4,1,903-1,0,0,0,0,0,8,0,0,...,0,0,0,0,Train Wheels and Couplers,1969,371,10,Supplemental,365.0
5,1,850950-1,0,15,1,0,0,0,0,0,...,0,0,0,0,Christmas Cat Ornament,2014,228,34,Creator,227.0
6,1,4444-1,0,2,0,0,0,1,0,0,...,0,0,0,0,Coca-Cola Defender 2,2002,462,4,Soccer,458.0
7,1,3474-1,0,0,0,0,0,0,0,0,...,0,0,0,0,1 x 6 Light Gray Bricks,2000,254,50,Bulk Bricks,NaN
8,1,30277-1,0,8,0,0,0,0,0,0,...,0,0,0,0,First Order Star Destroyer,2016,184,56,Star Wars Episode 7,158.0
9,1,71012-11,0,1,0,0,0,0,3,0,...,0,0,0,0,Minnie Mouse,2016,555,6,Disney,535.0


In [ ]:
from sklearn.tree import DecisionTreeClassifier

#Now choose a random subset of that for training/testing
np.random.seed=4 #comment out for fun reruns, used later
msk = np.random.rand(len(forprediction)) < 0.8
forprediction_train = forprediction[msk]
forprediction_test = forprediction[~msk]

#Take away the answers
y_train = forprediction_train['quintile_id']
y_test = members_small_test['quintile_id']
x_train = members_small_train.drop('quintile_id', axis = 1)

#x_train = x_train.drop('targetcost', axis = 1)
x_test = members_small_test.drop('quintile_id', axis = 1)
#x_test = x_test.drop('targetcost', axis = 1)

#Make the tree
dt = DecisionTreeClassifier(min_samples_split=1600)
dt.fit(x_train, y_train)

#Add predictions to original stuff
costbins_pred = list(dt.predict(x_test))

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, costbins_pred)
matrix

In [ ]:
inventories.set_index('set_num')

In [ ]:
inventories.join(sets)

This could now be merged with the sets dataframe and it would be sufficient, but there's something really weird going on with this data that needs to be explored more:

Run the 2nd and 3rd cells in this notebook again before running these, otherwise you'll get a bunch of keyerrors.

In [ ]:
#all of the entires in inventories have unique inventory ids - no inventory shows up in more than one set
#some sets do have more than one inventory
print(len(inventories))
print(len(inventories['id'].unique())) #I set id to the index above, whoops.  Reload the data from the begining and uncomment
print(len(inventories['set_num'].unique()))

In [ ]:
#inventory_sets has inventories that are in more than one set, not very many over all
print len(inventory_sets)
print len(inventory_sets['set_num'].unique())
print len(inventory_sets['inventory_id'].unique())
print len(inventory_parts['inventory_id'].unique())

In [ ]:
#the total number of sets is slightly larger than the number of unique sets showing up in the inventories df
print(len(sets))
print(len(inventories['set_num'].unique()))

In [ ]:
#There also appear to be inventories that have no entries in inventory_parts - no parts in those?
print len(inventory_parts)
print 'unique inventories in inventory_parts: ', len(inventory_parts['inventory_id'].unique())
print 'unique inventories in inventories: ', len(inventories['id'].unique())
print 'unique inventories in inventories_sets: ', len(inventory_sets['inventory_id'].unique())

In [ ]:
#all of the parts and sets entries are in inventories:
ip = set(inventory_parts['inventory_id'].unique())
i_s = set(inventory_sets['inventory_id'].unique())
i = set(inventories['id'].unique())
print len(i.difference(ip)) #number of things in i that aren't in ip
print len(inventories['id'].unique()) - len(inventory_parts['inventory_id'].unique())
print len(i.difference(i_s))
print len(inventories['id'].unique()) - len(inventory_sets['inventory_id'].unique())

In [ ]:
#This prints out all the set information for all the inventories that don't entries in inventory_parts
#A lot of it is swag, but some things list that they have parts!
inventory_parts
ip = set(inventory_parts['inventory_id'].unique())
i = set(inventories['id'].unique())

inventories_not_in_parts = i.difference(ip)
#print inventories_not_in_parts

for x in inventories_not_in_parts:
    weird_inv = inventories[inventories['id'] == x]
    print sets.loc[weird_inv.iloc[0]['set_num']]
